####Libraries
***

In [1]:
import sklearn
import seaborn as sns
import pandas as pd
import numpy as np
import seaborn as sns
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()
# pip install TA-lib
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz

!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=$HOME
!make
!make install

!TA_LIBRARY_PATH=~/lib TA_INCLUDE_PATH=~/include pip install ta-lib
import talib as ta
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 5.5 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
--2022-09-16 14:33:26--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2022-09-16 14:33:26--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownlo

#### Gradient Boosting Next Day 
***

#####Load Dataset
***

In [2]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   #change fb to meta 25/08/2022
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-03-01','2022-06-01')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο 
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-03-01','2022-06-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd1= rd.pct_change().shift(-1)
rd1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,-0.005166,-0.003389,0.002692,0.009758,-0.038855,-0.014362,-0.016404,0.006779,-0.004723,0.003893,...,0.004514,-0.016226,-0.002967,-0.016730,-0.024301,-0.000030,-0.029984,-0.003970,-0.003951,-0.005851
2021-03-02,-0.010570,-0.001700,0.011055,-0.005674,-0.017532,-0.009301,-0.028932,0.017843,0.007966,0.023986,...,0.003932,-0.052491,0.001281,-0.031045,-0.056018,-0.005289,0.005146,-0.004264,-0.009436,0.008026
2021-03-03,-0.009933,0.012185,0.000312,-0.015604,-0.062370,-0.028835,-0.009128,-0.018467,-0.034918,0.000631,...,-0.008951,-0.062691,-0.010527,-0.017252,0.008735,0.002824,-0.021463,-0.015592,-0.019174,0.038747
2021-03-04,0.009938,0.021359,0.029981,0.026359,0.048134,0.023292,0.007687,-0.002247,0.029502,0.032591,...,0.023597,-0.000084,0.030124,0.032602,0.009909,0.039813,0.042827,0.018487,0.040025,0.037813
2021-03-05,-0.005529,0.020152,0.014251,0.028065,-0.068488,0.007962,-0.016167,0.035238,0.032549,0.019955,...,0.022722,-0.054215,0.016768,-0.013661,0.063997,0.008845,0.005449,0.022562,0.020997,-0.000985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-24,0.019113,0.002544,0.002877,0.000833,0.020690,0.017726,0.025696,-0.000341,0.015476,0.031356,...,0.012588,0.018690,0.001752,0.003409,0.048319,0.001065,0.030179,0.005971,-0.001287,0.020127
2022-05-25,-0.009147,0.000976,-0.006730,0.024792,0.060260,0.034232,0.040295,0.002488,0.021167,0.030279,...,0.023493,0.003744,0.017455,0.026331,0.069199,0.008312,0.032702,0.023106,0.017514,0.003531
2022-05-26,-0.003786,0.009651,0.011373,0.024914,0.038325,0.015164,0.036632,0.026576,0.028191,0.025579,...,0.016354,0.059562,0.019985,0.009931,0.003679,0.009717,0.044306,0.020762,0.028438,0.009830


In [3]:
rd2=rd1.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,25,30,39,45,1,12,10,42,26,40,...,41,11,33,9,4,37,3,28,29,21
2021-03-02,16,28,36,24,11,18,6,39,33,43,...,31,2,30,4,1,25,32,27,17,34
2021-03-03,24,40,32,19,2,6,25,16,5,33,...,26,1,23,17,37,34,11,20,12,45
2021-03-04,8,16,28,23,43,19,6,1,27,32,...,20,3,29,33,7,37,41,13,39,34
2021-03-05,9,34,28,41,1,22,7,43,42,33,...,38,2,29,8,45,24,19,37,35,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,32,44,39,19,6,15,4,25,9,10,...,17,5,27,21,12,37,8,7,22,30
2022-05-24,37,15,17,11,39,34,40,9,30,43,...,25,36,14,19,44,12,41,22,7,38
2022-05-25,2,4,3,28,41,37,38,6,23,33,...,27,8,19,30,42,12,36,25,20,7


In [4]:
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,3,4,5,5,1,2,2,5,3,5,...,5,2,4,1,1,5,1,4,4,3
2021-03-02,2,4,4,3,2,2,1,5,4,5,...,4,1,4,1,1,3,4,3,2,4
2021-03-03,3,5,4,3,1,1,3,2,1,4,...,3,1,3,2,5,4,2,3,2,5
2021-03-04,1,2,4,3,5,3,1,1,3,4,...,3,1,4,4,1,5,5,2,5,4
2021-03-05,1,4,4,5,1,3,1,5,5,4,...,5,1,4,1,5,3,3,5,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,4,5,5,3,1,2,1,3,1,2,...,2,1,3,3,2,5,1,1,3,4
2022-05-24,5,2,2,2,5,4,5,1,4,5,...,3,4,2,3,5,2,5,3,1,5
2022-05-25,1,1,1,4,5,5,5,1,3,4,...,3,1,3,4,5,2,4,3,3,1


#####Features
***

In [5]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Μάρτιο 2022
  stock[i]=pdr.get_data_yahoo(i,'2021-03-01','2022-06-01') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk[j].astype(int).shift(1)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Functions
***

In [7]:
def RPS_calculation(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [8]:
def create_historical(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [9]:
def predictions(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      # print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-59] #only the days of one year until 3/01
      
      Y_train= y.iloc[:-59]

      print(Y_train.tail(1))
      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      for i in range(58,0,-1): #from 4/01 and later -->predictions
        X_validation=X.iloc[-(i+1):-i] 
        print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        # print(Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      X_validation=X.tail(1) 
      # print(X_validation)
      Y_validation= y.tail(1)
      print(Y_validation.index) 
      probability= model.predict_proba(X_validation)
      print(probability[0])
      tmp.append(probability[0])
      prob[j]=tmp
  return prob

In [10]:
def RPS_data(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [11]:
def create_RPS(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

#####Gradient Boosting (baseline)
***

In [12]:
from sklearn.ensemble import GradientBoostingClassifier
baseline= GradientBoostingClassifier()
#---predictions---#
prob1=predictions(baseline,assets,stock,29)
#---probabilities---#
prob1['date']=stock['ABBV'].iloc[-59:].index
prob1_final=pd.DataFrame.from_dict(prob1)
prob1_final.set_index('date')
RPS1= RPS_data(prob1_final,rd)
RPS1= create_RPS(prob1_final,RPS1)
RPS1
# print(RPS1.mean(axis=1).mean())

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.
2022-05-18 -6.904307 -1.906449 -12.747588           18.0  

[1 rows x 24 columns]
            (open2-open1)/open2  (high2-high1)/high2  (low2-low1)/low2  \
Date                                                                     
2022-05-19             0.011619             0.011138           0.04115   

            (Adj Close2-Adj Close1)/Adj Close2      av_3      av_6     av_10  \
Date                                                                           
2022-05-19                            0.042942 -0.009556 -0.005695 -0.007383   

              av_14  high1-low1/high1  close1-open1/close1  ...   TRIX_14  \
Date                                                        ...             
2022-05-19 -0.00764           0.04262             -0.03581  ... -0.241459   

             TRIX_10     WILLR   std_dev  dist_from_mean_10  dist_from_mean_6  \
Date                                                                            
2022-

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


            (open2-open1)/open2  (high2-high1)/high2  (low2-low1)/low2  \
Date                                                                     
2022-05-26            -0.018961            -0.018356         -0.023289   

            (Adj Close2-Adj Close1)/Adj Close2      av_3      av_6     av_10  \
Date                                                                           
2022-05-26                           -0.020127  0.015849  0.007562  0.011467   

               av_14  high1-low1/high1  close1-open1/close1  ...   TRIX_14  \
Date                                                         ...             
2022-05-26  0.005507           0.02134             0.017861  ...  0.328226   

             TRIX_10     WILLR   std_dev  dist_from_mean_10  dist_from_mean_6  \
Date                                                                            
2022-05-26  0.448036 -0.017517  2.813145           4.609216          3.228629   

                MACD       PPO     CMO_14  absolute rank 

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-07,0.212124,0.401227,0.151706,0.030198,0.001007,0.159472,0.221916,0.385162,0.260030,0.139977,...,0.191736,0.381148,0.161306,0.004103,0.199058,0.258019,0.003873,0.243591,0.312572,0.088290
2022-03-08,0.134590,0.532451,0.071224,0.187587,0.407262,0.168504,0.087983,0.016483,0.358801,0.386046,...,0.224685,0.558397,0.271167,0.088281,0.057257,0.021189,0.109928,0.395691,0.162805,0.207130
2022-03-09,0.117594,0.143057,0.089587,0.460766,0.203137,0.195328,0.346893,0.104913,0.038546,0.135539,...,0.074579,0.001912,0.019953,0.140222,0.076709,0.319326,0.459664,0.142704,0.032408,0.133308
2022-03-10,0.382348,0.253945,0.146074,0.144239,0.119941,0.050398,0.027494,0.038245,0.169303,0.131138,...,0.600946,0.005933,0.223522,0.274048,0.031093,0.059265,0.758023,0.140798,0.136268,0.183158
2022-03-11,0.228254,0.164526,0.068936,0.078511,0.452122,0.189177,0.258112,0.207498,0.588963,0.002767,...,0.140733,0.187927,0.029257,0.089457,0.216469,0.133364,0.272752,0.539790,0.419764,0.467871
2022-03-14,0.100680,0.263260,0.181514,0.089753,0.264797,0.046868,0.223493,0.310853,0.151376,0.254339,...,0.237517,0.564910,0.259156,0.002598,0.266241,0.059977,0.476768,0.588962,0.239920,0.148157
2022-03-15,0.169599,0.422773,0.047054,0.104903,0.171402,0.223601,0.268895,0.452511,0.165294,0.323209,...,0.324021,0.243768,0.019961,0.133316,0.336168,0.327880,0.188344,0.127754,0.191260,0.769997
2022-03-16,0.023893,0.538608,0.179530,0.108212,0.050824,0.000439,0.472944,0.255510,0.086686,0.119852,...,0.079632,0.393546,0.146537,0.266546,0.109383,0.173252,0.215880,0.122100,0.275510,0.002777
2022-03-17,0.135086,0.409955,0.169297,0.148553,0.225924,0.104073,0.234403,0.008819,0.251644,0.120899,...,0.378686,0.759020,0.103263,0.130060,0.355219,0.339551,0.115699,0.137741,0.039805,0.535321


In [13]:
RPS1

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-07,0.212124,0.401227,0.151706,0.030198,0.001007,0.159472,0.221916,0.385162,0.260030,0.139977,...,0.191736,0.381148,0.161306,0.004103,0.199058,0.258019,0.003873,0.243591,0.312572,0.088290
2022-03-08,0.134590,0.532451,0.071224,0.187587,0.407262,0.168504,0.087983,0.016483,0.358801,0.386046,...,0.224685,0.558397,0.271167,0.088281,0.057257,0.021189,0.109928,0.395691,0.162805,0.207130
2022-03-09,0.117594,0.143057,0.089587,0.460766,0.203137,0.195328,0.346893,0.104913,0.038546,0.135539,...,0.074579,0.001912,0.019953,0.140222,0.076709,0.319326,0.459664,0.142704,0.032408,0.133308
2022-03-10,0.382348,0.253945,0.146074,0.144239,0.119941,0.050398,0.027494,0.038245,0.169303,0.131138,...,0.600946,0.005933,0.223522,0.274048,0.031093,0.059265,0.758023,0.140798,0.136268,0.183158
2022-03-11,0.228254,0.164526,0.068936,0.078511,0.452122,0.189177,0.258112,0.207498,0.588963,0.002767,...,0.140733,0.187927,0.029257,0.089457,0.216469,0.133364,0.272752,0.539790,0.419764,0.467871
2022-03-14,0.100680,0.263260,0.181514,0.089753,0.264797,0.046868,0.223493,0.310853,0.151376,0.254339,...,0.237517,0.564910,0.259156,0.002598,0.266241,0.059977,0.476768,0.588962,0.239920,0.148157
2022-03-15,0.169599,0.422773,0.047054,0.104903,0.171402,0.223601,0.268895,0.452511,0.165294,0.323209,...,0.324021,0.243768,0.019961,0.133316,0.336168,0.327880,0.188344,0.127754,0.191260,0.769997
2022-03-16,0.023893,0.538608,0.179530,0.108212,0.050824,0.000439,0.472944,0.255510,0.086686,0.119852,...,0.079632,0.393546,0.146537,0.266546,0.109383,0.173252,0.215880,0.122100,0.275510,0.002777
2022-03-17,0.135086,0.409955,0.169297,0.148553,0.225924,0.104073,0.234403,0.008819,0.251644,0.120899,...,0.378686,0.759020,0.103263,0.130060,0.355219,0.339551,0.115699,0.137741,0.039805,0.535321


In [14]:
print(RPS1.mean(axis=1).mean())

0.20229386109830705


In [15]:
print(RPS1.mean(axis=1))

date
2022-03-07    0.195995
2022-03-08    0.221663
2022-03-09    0.181925
2022-03-10    0.204846
2022-03-11    0.218597
2022-03-14    0.191024
2022-03-15    0.252201
2022-03-16    0.200604
2022-03-17    0.204535
2022-03-18    0.181492
2022-03-21    0.217080
2022-03-22    0.194032
2022-03-23    0.221825
2022-03-24    0.170604
2022-03-25    0.248553
2022-03-28    0.202347
2022-03-29    0.177977
2022-03-30    0.162229
2022-03-31    0.191502
2022-04-01    0.208436
2022-04-04    0.202201
2022-04-05    0.192354
2022-04-06    0.161499
2022-04-07    0.163959
2022-04-08    0.189517
2022-04-11    0.194142
2022-04-12    0.186073
2022-04-13    0.204445
2022-04-14    0.186227
2022-04-18    0.206507
2022-04-19    0.216606
2022-04-20    0.223806
2022-04-21    0.227054
2022-04-22    0.216664
2022-04-25    0.213369
2022-04-26    0.219870
2022-04-27    0.203619
2022-04-28    0.167455
2022-04-29    0.189008
2022-05-02    0.178773
2022-05-03    0.200310
2022-05-04    0.181751
2022-05-05    0.216456
2022-0

#####Gradient Boosting (with feature selection and tuning) 
***

In [16]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [18]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob2=predictions(model1,assets,stock1,21)
#---probabilities---#
prob2['date']=stock['ABBV'].iloc[-59:].index
prob2_final=pd.DataFrame.from_dict(prob2)
prob2_final.set_index('date')
RPS2= RPS_data(prob2_final,rd)
RPS2= create_RPS(prob2_final,RPS2)
RPS2
# print(RPS2.mean(axis=1).mean())

Η έξοδος ροής περικόπηκε στις τελευταίες 5000 γραμμές.

                 ADX    ADX_14   TRIX_14   TRIX_10     WILLR   std_dev  \
Date                                                                     
2022-03-29  0.172815  0.180458 -0.158574 -0.010655 -0.310266  1.578722   

            dist_from_mean_10       PPO  
Date                                     
2022-03-29           0.652242  0.821993  
            (open2-open1)/open2  (low2-low1)/low2      av_3      av_6  \
Date                                                                    
2022-03-30            -0.031551         -0.041834  0.018846  0.008225   

               av_14  high1-low1/high1  close1-open1/close1       RSI  \
Date                                                                    
2022-03-30  0.005789          0.029185             0.024804  0.730061   

                 ADX    ADX_14   TRIX_14   TRIX_10     WILLR   std_dev  \
Date                                                                     
2022-03

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-07,0.140801,0.130958,0.063267,0.241338,0.202571,0.072999,0.095472,0.140192,0.128673,0.120689,...,0.210109,0.256398,0.123651,0.242734,0.265334,0.286412,0.229508,0.122048,0.061761,0.128889
2022-03-08,0.140346,0.267919,0.060879,0.120803,0.133655,0.231486,0.094870,0.080329,0.277180,0.240204,...,0.107658,0.256398,0.131227,0.242734,0.118761,0.069787,0.109303,0.251934,0.129123,0.241712
2022-03-09,0.101383,0.116183,0.112608,0.234507,0.232500,0.072999,0.251526,0.080108,0.108996,0.120689,...,0.082002,0.177030,0.083067,0.086862,0.217316,0.208806,0.223526,0.116700,0.129123,0.221971
2022-03-10,0.206588,0.110753,0.264693,0.110740,0.116346,0.101645,0.093906,0.080296,0.129904,0.073060,...,0.241927,0.157348,0.086122,0.129810,0.199639,0.138835,0.252680,0.125280,0.124507,0.121619
2022-03-11,0.206588,0.121790,0.060879,0.076879,0.232500,0.072999,0.209452,0.140192,0.287350,0.235029,...,0.210109,0.121729,0.083067,0.224716,0.207017,0.100381,0.124181,0.255607,0.255197,0.232048
2022-03-14,0.101383,0.125459,0.060879,0.120502,0.203118,0.101645,0.253283,0.273348,0.232411,0.073100,...,0.259654,0.256321,0.235873,0.241597,0.094670,0.069787,0.159093,0.255607,0.108852,0.232048
2022-03-15,0.140801,0.260530,0.110569,0.076882,0.203118,0.231486,0.139730,0.137798,0.065034,0.073100,...,0.119217,0.256321,0.083067,0.128971,0.256215,0.286412,0.148161,0.127178,0.062658,0.250529
2022-03-16,0.101018,0.260530,0.063868,0.077588,0.116346,0.101645,0.257753,0.079928,0.235070,0.232302,...,0.077723,0.256321,0.120977,0.118322,0.109263,0.100381,0.237075,0.113398,0.255197,0.194559
2022-03-17,0.083009,0.260530,0.117328,0.112957,0.204164,0.073770,0.257753,0.279362,0.104409,0.232302,...,0.246510,0.281408,0.087218,0.241597,0.134750,0.286412,0.185232,0.269599,0.255197,0.232888


In [19]:
print(RPS2.mean(axis=1).mean())

0.15829018740488784


In [20]:
print(RPS2.mean(axis=1))

date
2022-03-07    0.159516
2022-03-08    0.162456
2022-03-09    0.153710
2022-03-10    0.158143
2022-03-11    0.156566
2022-03-14    0.159578
2022-03-15    0.163249
2022-03-16    0.158021
2022-03-17    0.163106
2022-03-18    0.153626
2022-03-21    0.163177
2022-03-22    0.151727
2022-03-23    0.162960
2022-03-24    0.153767
2022-03-25    0.161640
2022-03-28    0.163224
2022-03-29    0.150672
2022-03-30    0.159924
2022-03-31    0.160676
2022-04-01    0.164167
2022-04-04    0.151987
2022-04-05    0.154238
2022-04-06    0.153436
2022-04-07    0.154089
2022-04-08    0.165037
2022-04-11    0.155234
2022-04-12    0.161961
2022-04-13    0.155920
2022-04-14    0.155138
2022-04-18    0.163303
2022-04-19    0.157922
2022-04-20    0.155755
2022-04-21    0.160575
2022-04-22    0.159992
2022-04-25    0.154841
2022-04-26    0.156594
2022-04-27    0.160657
2022-04-28    0.161132
2022-04-29    0.162161
2022-05-02    0.153445
2022-05-03    0.159796
2022-05-04    0.152248
2022-05-05    0.152495
2022-0

In [21]:
#save the results 
RPS2.to_excel('Q1-Gradient_Boosting_next_day.xlsx')

####Gradient Boosting next month
***

#####Load Dataset 
***

In [22]:
#φτιάχνω τη λίστα με τα assets μου -> 50 μετοχές 
#διόρθωση : θα βγάλω τυχαία άλλες 4 μετοχές μαζί με την OGN επειδή δεν έχει το ίδιο μέγεθος με τις άλλες 
#βγάζω OGN ,WRK,HST,CE,ACN
assets= ["ABBV","AEP","AIZ","ALLE","AMAT","AMP","AMZN","AVB","AVY",   
  "AXP","BDX","BF-B","BMY","BR","CARR","CDW","CHTR","CNC",   
  "CNP","COP","CTAS","CZR","DG","DPZ","DRE","DXC","META","FTV",   
  "GOOG","GPC","HIG","JPM","KR","PG","PPL","PRU",   
  "PYPL","RE","ROL","ROST","UNH","URI","V","VRSK","XOM"]
#άρα πρέπει να κατεβάσουμε όλα τα adj close για όλες τις τιμές όλων των μετοχών και να κάνουμε απόλυτο ranking και σε 5 κλασεις (targets)
#αλλαγή σε παραπάνω μήνες (+3)
r=pd.DataFrame(index=pdr.get_data_yahoo(assets[0],'2021-03-01','2022-09-01')['Adj Close'].index)
#παίρνω τιμές για ένα χρόνο 
for i in assets:
  # print('i metoxi me onoma:',i,'exei megethos',pdr.get_data_yahoo(i,'2021-01-01','2022-04-01')['Adj Close'].count())
  temp= pdr.get_data_yahoo(i,'2021-03-01','2022-09-01')['Adj Close']
  temp=temp.dropna()
  r[i]=temp.values
#φτιάχνω ένα αντίγραφο με τις τιμές για να το έχω 
rd=pd.DataFrame(r)
rd1= rd.pct_change()
rd1

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-02,-0.005166,-0.003389,0.002692,0.009758,-0.038855,-0.014362,-0.016404,0.006779,-0.004723,0.003893,...,0.004514,-0.016226,-0.002967,-0.016730,-0.024301,-0.000030,-0.029984,-0.003970,-0.003951,-0.005851
2021-03-03,-0.010570,-0.001701,0.011055,-0.005674,-0.017532,-0.009301,-0.028932,0.017843,0.007966,0.023986,...,0.003932,-0.052491,0.001281,-0.031045,-0.056018,-0.005289,0.005146,-0.004264,-0.009436,0.008026
2021-03-04,-0.009933,0.012186,0.000312,-0.015604,-0.062370,-0.028834,-0.009128,-0.018467,-0.034918,0.000631,...,-0.008952,-0.062691,-0.010527,-0.017252,0.008735,0.002824,-0.021463,-0.015592,-0.019174,0.038747
2021-03-05,0.009938,0.021359,0.029981,0.026359,0.048134,0.023292,0.007687,-0.002247,0.029502,0.032590,...,0.023597,-0.000084,0.030124,0.032602,0.009909,0.039813,0.042827,0.018487,0.040025,0.037813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-25,0.010297,0.007013,0.007720,0.014234,0.034687,0.021279,0.026009,0.016288,0.010221,0.019481,...,0.018894,0.031570,0.011399,0.002248,0.003865,0.009525,0.037565,0.015242,0.003236,0.005173
2022-08-26,-0.021388,-0.015090,-0.021443,-0.054781,-0.059048,-0.031059,-0.047567,-0.019073,-0.035360,-0.030327,...,-0.028211,-0.041563,-0.014124,-0.033361,-0.040150,-0.022749,-0.041149,-0.033028,-0.027751,-0.012312
2022-08-29,-0.004694,0.010902,-0.003095,-0.009625,-0.022875,-0.013462,-0.007342,-0.017390,-0.014779,-0.017609,...,-0.005481,-0.000431,-0.002895,-0.012761,-0.003438,0.000983,-0.016938,-0.002760,-0.001896,0.022990


In [23]:
#prepare the ranks for next month (22 working days later)
rd1_next_month = rd1.shift(-22)
rd1_next_month

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,0.013391,-0.000354,-0.020317,0.002634,0.053960,-0.001503,0.012696,-0.015264,-0.008262,-0.011324,...,-0.010643,0.026634,-0.025483,0.014740,-0.003904,-0.003989,-0.011704,-0.000802,0.004206,-0.015170
2021-03-02,0.002772,0.009917,0.003739,0.016956,0.059282,0.012734,0.021628,0.012953,0.024068,0.022130,...,0.017673,0.019354,0.013317,0.011912,0.005004,-0.013438,0.007288,0.024229,0.017658,0.027942
2021-03-03,-0.021932,0.008768,0.003443,0.017847,0.010811,0.012191,0.020794,-0.004869,0.015313,0.010306,...,0.000432,0.014058,-0.000398,0.016078,0.020496,0.008472,-0.000543,0.010053,0.009677,-0.015508
2021-03-04,-0.007160,0.005563,0.004272,0.011305,-0.024537,-0.007764,-0.000902,0.002742,0.007122,0.000616,...,0.003127,0.008645,0.010239,-0.002826,0.004228,-0.015695,-0.018400,-0.001781,-0.003636,-0.002832
2021-03-05,-0.001613,-0.000922,0.015202,-0.001065,-0.002867,0.007021,0.017237,0.006863,-0.008892,0.006705,...,0.009781,0.009519,0.008597,-0.022953,-0.000729,0.003485,-0.010540,0.002836,-0.004975,0.004437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
rd2=rd1_next_month.rank(axis=1,method='first')
rd2=rd2.dropna()
rd2.astype(int)
jk=rd2.copy()
jk.astype(int)
def rank_data(df1):
  for i in range(len(df1.index)):
    d=df1.loc[df1.index[i]]  #της κάθε μέρας d[1] δίνει μόνο τις τιμές 
    #print(d)
    d=d.sort_values()
    d[0:9]=1
    d[9:18]=2
    d[18:27]=3
    d[27:36]=4
    d[36:45]=5
    df1.loc[df1.index[i]]=d
    #print(df1.loc[df1.index[i]])
    #print(d)
  df1=df1.astype(int)
  return df1
r5=rd2.copy()
rd3=rank_data(r5)
rd3

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-01,5,4,1,4,5,3,5,1,3,2,...,2,5,1,5,3,3,2,3,4,1
2021-03-02,2,3,2,4,5,3,4,3,5,5,...,4,4,3,3,2,1,2,5,4,5
2021-03-03,1,2,2,5,3,3,5,1,4,3,...,2,4,1,4,5,2,1,3,3,1
2021-03-04,2,4,4,5,1,2,3,4,5,3,...,4,5,5,3,4,1,1,3,2,2
2021-03-05,2,2,5,2,2,4,5,4,1,4,...,5,5,5,1,3,3,1,3,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-26,3,2,2,3,5,4,5,4,3,4,...,4,5,3,1,2,2,5,3,1,2
2022-07-27,4,5,4,1,1,3,1,5,2,3,...,3,2,5,2,2,4,2,3,3,5
2022-07-28,3,5,3,2,1,2,3,1,1,1,...,3,4,4,2,3,4,1,4,4,5


In [25]:
#for absolute rank feature to use 
rd2_feature=rd1.rank(axis=1,method='first')
rd2_feature=rd2_feature.dropna()
rd2_feature.astype(int)
jk_feature=rd2_feature.copy()
jk_feature.astype(int)

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,25,30,39,45,1,12,10,42,26,40,...,41,11,33,9,4,37,3,28,29,21
2021-03-03,16,28,36,24,11,18,6,39,33,43,...,31,2,30,4,1,25,32,27,17,34
2021-03-04,24,40,32,19,2,6,25,16,5,33,...,26,1,23,17,37,34,11,20,12,45
2021-03-05,8,16,28,23,43,19,6,1,27,32,...,20,3,29,33,7,37,41,13,39,34
2021-03-08,9,34,28,41,1,22,7,43,42,33,...,38,2,29,8,45,24,19,37,35,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-25,21,14,15,26,44,35,40,30,20,34,...,32,42,22,6,10,17,45,27,9,12
2022-08-26,34,40,33,3,2,22,7,37,16,24,...,26,10,41,18,14,31,12,19,27,43
2022-08-29,23,42,27,13,3,10,19,6,9,4,...,21,34,28,11,26,35,7,29,31,45


#####Features 
All of them 
***

In [26]:
stock=dict()
for i in assets:
  #παίρνω δεδομένα για 15 μήνες περιπου->από Ιανουάριο 2021 μέχρι Iούνιο 2022 (3 μήνες παραπάνω δεδομένα για να τα χρησιμοποιήσουμε για τα target values)->προβλεψη τον επομενο μηνα
  stock[i]=pdr.get_data_yahoo(i,'2021-01-01','2022-09-01') 
  # print(df)
  #από αυτές κρατάμε μόνο τις πρώτες 5 στήλες 
  stock[i]=stock[i].iloc[:,:5]
stock

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

{'ABBV':                   Open        High         Low       Close   Adj Close
 Date                                                                  
 2021-01-04  107.180000  107.349998  103.860001  105.410004   97.798592
 2021-01-05  105.410004  107.019997  104.629997  106.500000   98.809891
 2021-01-06  104.750000  107.190002  104.180000  105.580002   97.956322
 2021-01-07  106.110001  107.059998  105.570000  106.709999   99.004715
 2021-01-08  106.839996  107.529999  105.760002  107.269997   99.524277
 ...                ...         ...         ...         ...         ...
 2022-08-25  138.770004  139.559998  137.669998  139.330002  139.330002
 2022-08-26  138.889999  139.850006  136.250000  136.350006  136.350006
 2022-08-29  136.199997  136.720001  134.759995  135.710007  135.710007
 2022-08-30  135.139999  136.490005  134.889999  135.550003  135.550003
 2022-08-31  135.690002  136.039993  134.169998  134.460007  134.460007
 
 [419 rows x 5 columns],
 'AEP':                   Ope

In [27]:
for j in assets:
    df=stock[j] #all the days we need so we can fix the columns 
    # print('to df ftanei mexri ',stock[j].iloc[:-61].tail(1))
    # print(df)
    #τώρα θα προσθέσουμε τις καινούριες στήλες με τα features μας 
    df['(open2-open1)/open2']= (df['Open'].shift(2)-df['Open'].shift(1))/df['Open'].shift(2)
    df['(high2-high1)/high2']= (df['High'].shift(2)-df['High'].shift(1))/df['High'].shift(2)
    df['(low2-low1)/low2']= (df['Low'].shift(2)-df['Low'].shift(1))/df['Low'].shift(2)
    df['(Adj Close2-Adj Close1)/Adj Close2']=(df['Adj Close'].shift(2)-df['Adj Close'].shift(1))/df['Adj Close'].shift(2)
    df['av_3']=df['Adj Close'].shift(1).rolling(window=3).mean().pct_change()
    df['av_6']=df['Adj Close'].shift(1).rolling(window=6).mean().pct_change()
    df['av_10']=df['Adj Close'].shift(1).rolling(window=10).mean().pct_change()
    df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    # df['av_14']=df['Adj Close'].shift(1).rolling(window=14).mean().pct_change()
    df['high1-low1/high1']=(df['High'].shift(1)-df['Low'].shift(1))/df['High'].shift(1)
    df['close1-open1/close1']=(df['Close'].shift(1)-df['Open'].shift(1))/df['Close'].shift(1)
    df['RSI']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=6)/100
    df['ADX'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=6)/100
    df['RSI_14']= ta.RSI(np.array(df['Adj Close'].shift(1)),timeperiod=14)/100
    df['ADX_14'] = ta.ADX(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1), timeperiod=14)/100
    df['TRIX_14'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=14)
    df['TRIX_10'] =ta.TRIX(df['Adj Close'].shift(1),timeperiod=10)
    df['WILLR']= ta.WILLR(df['High'].shift(1), df['Low'].shift(1), df['Close'].shift(1),timeperiod=14)/100
    df['std_dev']=df['Adj Close'].rolling(window=10).std().shift(1)
    df['dist_from_mean_10'] = (df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=10).mean()))
    df['dist_from_mean_6'] = df['Adj Close'].shift(1) - (df['Adj Close'].shift(1).rolling(window=6).mean())
    df['MACD']= ta.MACD(df['Adj Close'].shift(1),fastperiod=5,slowperiod=14, signalperiod=8)[0]
    df['PPO']= ta.PPO(df['Adj Close'].shift(1), fastperiod= 5,slowperiod=14,matype=0)
    df['CMO_14']= ta.CMO(df['Adj Close'].shift(1),timeperiod=14)
    df = df.iloc[1: , :]
    df['absolute rank']= jk_feature[j].astype(int).shift(1)

    ####CHANGE THE RANK TO NEXT MONTH --> 22 days later (working)
    df['Rank']= rd3[j]
    df1=df.dropna()
    stock[j]=df1.copy()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#####Functions
***

In [28]:
def RPS_calculation2(hist_data, submission, asset_no=44):
    # print('check')
    if hist_data.shape[0]<=asset_no:
        return np.nan    
    # print('check')
    asset_id = pd.unique(hist_data.symbol)
    
    for i in range(len(pd.unique(hist_data.date))):
        if len(hist_data[hist_data.date == pd.unique(hist_data.date)[i]])<len(asset_id):
            for asset in [x for x in asset_id if x not in hist_data[hist_data.date == pd.unique(hist_data.date)[i]].symbol.values]:
                right_price = hist_data[hist_data.symbol==asset].sort_values(by='date')
                right_price = right_price[right_price.date <= pd.unique(hist_data.date)[i]]
                right_price = right_price.price.iloc[-1]
                hist_data = hist_data.append({'date' : pd.unique(hist_data.date)[i],
                                               'symbol' : asset,
                                               'price' : right_price}, ignore_index=True)

    #Compute percentage returns
    asset_id = sorted(asset_id) 

    #Compute percentage returns
    returns = pd.DataFrame(columns = ["ID", "Return"])
    
    min_date = min(hist_data.date)
    max_date = max(hist_data.date)
    
    for i in range(0,len(asset_id)):
        temp = hist_data.loc[hist_data.symbol==asset_id[i]]
        
        open_price = float(temp.loc[temp.date==min_date].price)
        close_price = float(temp.loc[temp.date==max_date].price)
        
        returns = returns.append({'ID': temp.symbol.iloc[0], 
                                'Return': (close_price - open_price)/open_price}, ignore_index=True)

    #Define the relevant position of each asset
    ranking = pd.DataFrame(columns=["ID", "Position", "Return"])
    ranking.ID = list(asset_id)
    ranking.Return = returns.Return
    ranking.Position = ranking.Return.rank(method = 'min')

    #Handle Ties
    Series_per_position = pd.DataFrame(columns=["Position","Series", "Rank", "Rank1", "Rank2","Rank3", "Rank4", "Rank5"])
    Series_per_position.Position = list(pd.unique(ranking.Position.sort_values(ascending=True)))
    temp = ranking.Position.value_counts()
    temp = pd.DataFrame(zip(temp.index, temp), columns = ["Rank", "Occurencies"])
    temp = temp.sort_values(by = ["Rank"],ascending=True)
    Series_per_position.Series = list(temp.Occurencies)
    Series_per_position

    total_ranks = Series_per_position.Position.values[-1]
    for i in range(0,Series_per_position.shape[0]):
    
        start_p = Series_per_position.Position[i]
        end_p = Series_per_position.Position[i] + Series_per_position.Series[i]
        temp = pd.DataFrame(columns = ["Position","Rank", "Rank1", "Rank2", "Rank3", "Rank4","Rank5"])
        temp.Position = list(range(int(start_p),int(end_p)))

        if(temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank=1)
            temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(1,int(0.2*total_ranks+1))))].assign(Rank1=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank=2)
            temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.2*total_ranks+1),int(0.4*total_ranks+1))))].assign(Rank2=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank=3)
            temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.4*total_ranks+1),int(0.6*total_ranks+1))))].assign(Rank3=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank=4)
            temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.6*total_ranks+1),int(0.8*total_ranks+1))))].assign(Rank4=1.0)

        elif(temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].empty==False):
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank=5)
            temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))] = temp.loc[temp.Position.isin(list(range(int(0.8*total_ranks+1),int(total_ranks+1))))].assign(Rank5=1.0)
        temp = temp.fillna(0)
        Series_per_position.iloc[i,2:Series_per_position.shape[1]] = temp.mean(axis = 0).iloc[1:temp.shape[1]]

    Series_per_position = Series_per_position.drop('Series', axis = 1)
    ranking = pd.merge(ranking,Series_per_position, on = "Position")
    ranking = ranking[["ID", "Return", "Position", "Rank", "Rank1", "Rank2", "Rank3", "Rank4", "Rank5"]]
    ranking = ranking.sort_values(["Position"])
    # print(ranking)
    #Evaluate submission
    rps_sub = []
    #for aid in list((pd.unique(ranking.ID))):
    for aid in asset_id:

        target = np.cumsum(ranking.loc[ranking.ID==aid].iloc[:,4:9].values).tolist()
        frc = np.cumsum(submission.loc[submission.ID==aid].iloc[:,1:6].values).tolist()
        rps_sub.append(np.mean([(a - b)**2 for a, b in zip(target, frc)]))
    submission["RPS"] = rps_sub
    
    output = {'RPS' : np.mean(rps_sub),
              'details' : submission}
    
    return(output)

In [29]:
def create_historical2(prob):
  prob=prob.rename_axis("symbol", axis="columns")
  prob=prob.T
  prob['date']=prob.columns.date[0]
  prob.columns = ['price', 'date']
  prob=prob.reset_index()
  prob=prob[['symbol','date','price']]
  return prob

In [30]:
def predictions2(model,assets,stock,k):
  prob=dict()

  for j in assets:
      tmp=[]
      df=stock[j]
      df1=df.dropna()
      X= df1.iloc[:,5:k]
      #print(X) #it has all the days
      y= df1['Rank']
      X_train= X.iloc[:-105] #only the days of one year until 3/01--->CHANGE FROM -60 TO -100

      # print("this is the training",X_train.tail(1))

      Y_train= y.iloc[:-105]

      print("this is target training",Y_train.tail(1))

      model.fit(X_train,Y_train)
      #now each day we will use the models to predict 
      flag=True
      for i in range(104,42,-1): #from 4/01 and later -->predictions--->CHANGE FROM (59,0,-1) TO (99,39,-1)
        if(flag):
          first=X.iloc[-(i+1):-i].index
          print('this is the first day of predictions', first)
          flag=False
        X_validation=X.iloc[-(i+1):-i] 
        # print(X_validation)
        Y_validation= y.iloc[-(i+1):-i]
        print('this is for validation',Y_validation.index) 
        probability= model.predict_proba(X_validation)
        # print(probability[0])
        tmp.append(probability[0])
      last=X_validation.index
      print('this is last for prediction',last)
        #THAT WAS FOR LAST DAY -> NOT NECESSARY FOR THIS METHOD
      # X_validation=X.tail(1) 
      # print('this is last for prediction',X_validation)
      # Y_validation= y.tail(1)
      # print(Y_validation.index) 
      # probability= model.predict_proba(X_validation)
      # print(probability[0])
      # tmp.append(probability[0])
      prob[j]=tmp

      
  return prob ,first , last 

In [31]:
def RPS_data2(prob_final,rd):
  RPS=pd.DataFrame()

  # RPS=RPS.drop('date',axis=1)
  # print(RPS.index[2])
  for k in prob_final.index: #for everyday of a stock->index
    prob=pd.DataFrame()
    prob1=pd.DataFrame()
    predic=pd.DataFrame()
    if(prob_final['date'][k]!=prob_final['date'].tail(1).values): #if it's not for the last day 
      #for historical data of two days 
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==prob_final['date'][k+1])]
      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob : ',prob)
      mask1 = (rd.loc[(rd.index.date>=prob_final['date'][k+1])].index)
      prob1_table = rd.loc[mask1]
      prob1 = prob1_table.shift(-21).head(1)
      print('prob1 : ',prob1)
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)
      # print(RPS[RPS.index==k])

      
    else:
      #for the last day
      # prob=rd.loc[(rd.index.date==prob_final['date'][k])]
      # prob1=rd.loc[ (rd.index.date==rd.index.date[-1])]
      mask = (rd.loc[(rd.index.date>=prob_final['date'][k])].index)
      prob_table = rd.loc[mask]
      prob = prob_table.shift(-21).head(1)
      print('prob teleutaio :', prob)
      prob1 = pd.DataFrame(prob_table.iloc[1]).T
      print('prob1 : ',prob1)
      prob=create_historical(prob)
      prob1=create_historical(prob1)
      hist_data=prob.append(prob1)
      # print(hist_data)
      predic=prob_final.loc[prob_final['date']==prob_final['date'][k]] #find predictions of this day
      predic=predic.drop('date',axis=1)
      # print(predic)
      data=pd.DataFrame(index=predic.columns)
      data=data.reset_index()
      # print(data)
      data[['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=(0,0,0,0,0)
      for i in range(len(np.array_split(predic.values[0],46))-1):
        data.loc[i,['Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']]=np.array_split(predic.values[0],46)[i][0]
      data.columns = ['ID','Rank 1', 'Rank 2', 'Rank 3', 'Rank 4','Rank 5']
      # print(data)

      #now we have data->predictions and hist_data->historical data so now we can call the RPS
      rps=RPS_calculation(hist_data = hist_data , submission = data)['details']
      # print(rps) ->rps for every stock for one day 
      # print(rps['RPS'])
      tmp=pd.DataFrame(rps['RPS'].values).T
      # print(tmp.T)
      RPS=RPS.append(tmp,ignore_index=True)              
  return RPS

In [32]:
def create_RPS2(prob_final,RPS):
  d=prob_final['date']
  name=prob_final.columns[:-1]
  # print(np.split(name.values,len(name)))
  # print(d.values)
  RPS['date']=d.values
  RPS=RPS.set_index('date')
  RPS.columns=assets
  return RPS

#####Gradient Boosting (baseline)
***

In [34]:
from sklearn.ensemble import GradientBoostingClassifier
baseline2= GradientBoostingClassifier()
#---predictions---#
prob3, first , last = predictions2(baseline2,assets,stock,29)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob3['date']=stock['ABBV'].loc[mask].index
prob3_final=pd.DataFrame.from_dict(prob3)
prob3_final.set_index('date')
RPS3= RPS_data2(prob3_final,rd)
RPS3= create_RPS2(prob3_final,RPS3)
RPS3
# print(RPS3.mean(axis=1).mean())

this is target training Date
2022-03-01    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-03'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-04'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-07'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-08'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-09'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-10'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-11'], dtype='datetime64[ns]', name='Date', freq=None)
thi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18:

prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930489  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170898  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob1 :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630821  121.860001  293.727295  34.894276  92.495743   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.617981  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.105527,0.164878,0.485062,0.033330,0.424056,0.786490,0.062995,0.271440,0.076003,0.255005,...,0.442850,0.257867,0.191039,0.164818,0.102164,0.076782,0.150990,0.267575,0.264358,0.067225
2022-03-03,0.056319,0.313008,0.319240,0.104415,0.104298,0.005331,0.307111,0.309162,0.058782,0.324512,...,0.169026,0.161332,0.313171,0.029596,0.324164,0.164220,0.160359,0.135961,0.012517,0.067979
2022-03-04,0.280637,0.102953,0.129390,0.067248,0.371529,0.107316,0.155347,0.078886,0.111763,0.198173,...,0.117926,0.018744,0.078580,0.019354,0.116879,0.385031,0.633734,0.085602,0.294214,0.106195
2022-03-07,0.038113,0.457889,0.127802,0.232906,0.153084,0.270655,0.219264,0.235899,0.051175,0.270157,...,0.071890,0.026424,0.193028,0.171007,0.112730,0.392660,0.310466,0.150662,0.129608,0.123599
2022-03-08,0.132281,0.095155,0.020175,0.186007,0.651197,0.258407,0.505972,0.410141,0.267724,0.317056,...,0.220574,0.199782,0.240581,0.224355,0.131001,0.222929,0.691289,0.067535,0.054817,0.212115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,0.095480,0.415673,0.059560,0.136569,0.125967,0.152856,0.122222,0.429113,0.433234,0.141698,...,0.009192,0.513422,0.168585,0.277883,0.025572,0.523670,0.228370,0.119719,0.353833,0.229837
2022-05-24,0.056641,0.267604,0.114382,0.248129,0.248104,0.174539,0.237335,0.199302,0.146180,0.513387,...,0.264596,0.054476,0.210542,0.072807,0.253866,0.011009,0.220351,0.112857,0.015170,0.147886
2022-05-25,0.156569,0.188893,0.204229,0.062421,0.124759,0.196842,0.357066,0.006088,0.094286,0.129458,...,0.079094,0.008259,0.112188,0.009051,0.134740,0.049534,0.138020,0.045971,0.586833,0.074044


In [35]:
print(RPS3.mean(axis=1).mean())

0.19980144797080385


In [36]:
print(RPS3.mean(axis=1))

date
2022-03-02    0.211104
2022-03-03    0.213394
2022-03-04    0.188499
2022-03-07    0.169320
2022-03-08    0.212500
                ...   
2022-05-23    0.221049
2022-05-24    0.184019
2022-05-25    0.154954
2022-05-26    0.162870
2022-05-27    0.187146
Length: 62, dtype: float64


#####Gradient Boosting (with feature selection and tuning)
***

In [37]:
stock1=stock.copy()
for j in assets:
  stock1[j]=stock[j].drop(['dist_from_mean_6','RSI_14','av_10','(Adj Close2-Adj Close1)/Adj Close2','CMO_14','MACD','(high2-high1)/high2','absolute rank'],axis=1)

In [40]:
from sklearn.ensemble import GradientBoostingClassifier
model1= GradientBoostingClassifier(n_estimators=100,learning_rate=0.001,max_depth=1)
#---predictions---#
prob4 ,first,last =predictions2(model1,assets,stock1,21)
#---probabilities---#
mask = ((stock['ABBV'].index.values >= first.values) & (stock['ABBV'].index.values <= last.values))
prob4['date']=stock['ABBV'].loc[mask].index
prob4_final=pd.DataFrame.from_dict(prob4)
prob4_final.set_index('date')
RPS4= RPS_data2(prob4_final,rd)
RPS4= create_RPS2(prob4_final,RPS4)
RPS4
# print(RPS4.mean(axis=1).mean())

this is target training Date
2022-03-01    4.0
Name: Rank, dtype: float64
this is the first day of predictions DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-02'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-03'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-04'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-07'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-08'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-09'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-10'], dtype='datetime64[ns]', name='Date', freq=None)
this is for validation DatetimeIndex(['2022-03-11'], dtype='datetime64[ns]', name='Date', freq=None)
thi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18:

                  ABBV        AEP         AIZ        ALLE        AMAT  \
Date                                                                    
2022-03-03  159.814133  99.930489  183.961975  110.589905  126.775017   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-03  294.668579  163.559998  252.387024  173.579086  185.944336  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-03  115.603111  116.669998  303.419739  34.615841  89.954269   

                   UNH    URI           V        VRSK        XOM  
Date                                                              
2022-03-03  509.170898  354.0  225.529037  213.353729  81.485481  

[1 rows x 45 columns]
prob :                    ABBV        AEP         AIZ        ALLE        AMAT  \
Date                

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV       AEP         AIZ        ALLE        AMAT  \
Date                                                                   
2022-03-04  159.038071  99.08387  180.370453  110.371742  128.794907   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-03-04  294.787476  168.346497  248.984695  173.113022  186.431122  ...   

                   PRU        PYPL          RE        ROL       ROST  \
Date                                                                   
2022-03-04  113.630821  121.860001  293.727295  34.894276  92.495743   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-03-04  506.617981  349.899994  226.903992  216.233383  81.524696  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ        ALLE        AMAT  \

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-26  151.076859  94.338966  172.290344  97.975998  94.777618   

                   AMP        AMZN         AVB        AVY         AXP  ...  \
Date                                                                   ...   
2022-05-26  242.633484  107.400002  195.470001  163.45636  141.659866  ...   

                  PRU   PYPL          RE        ROL       ROST        UNH  \
Date                                                                        
2022-05-26  96.546379  71.82  278.745331  34.418652  72.525093  506.84964   

                   URI           V        VRSK        XOM  
Date                                                       
2022-05-26  247.990005  197.769714  168.658813  90.619514  

[1 rows x 45 columns]
prob1 :                    ABBV        AEP         AIZ       ALLE       AMAT  \
Date                        

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.


prob teleutaio :                   ABBV        AEP         AIZ       ALLE       AMAT  \
Date                                                                  
2022-05-27  152.711563  94.626709  170.169083  97.966034  91.705482   

                   AMP        AMZN         AVB         AVY         AXP  ...  \
Date                                                                    ...   
2022-05-27  238.890869  108.919998  195.080002  163.336853  138.949997  ...   

                  PRU       PYPL          RE        ROL       ROST  \
Date                                                                 
2022-05-27  95.212051  71.470001  276.866699  34.877438  71.558487   

                   UNH         URI           V        VRSK        XOM  
Date                                                                   
2022-05-27  514.096924  241.070007  199.147278  171.034866  87.272049  

[1 rows x 45 columns]
prob1 :                    ABBV         AEP        AIZ        ALLE        AMAT  \

,ABBV,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,AXP,...,PRU,PYPL,RE,ROL,ROST,UNH,URI,V,VRSK,XOM
date,,,,,,,,,,,,,,,,,,,,,
2022-03-02,0.077226,0.243426,0.256568,0.127052,0.221800,0.275930,0.117441,0.230540,0.071090,0.136305,...,0.138127,0.122920,0.209571,0.234990,0.110765,0.069305,0.127771,0.255289,0.126540,0.102030
2022-03-03,0.142227,0.115780,0.271220,0.076490,0.141743,0.074119,0.243749,0.269860,0.108842,0.115994,...,0.259307,0.246164,0.263529,0.118663,0.261497,0.132608,0.211623,0.115235,0.108107,0.101921
2022-03-04,0.195507,0.113374,0.063622,0.103029,0.225698,0.074342,0.220312,0.076624,0.121632,0.124464,...,0.110821,0.183495,0.113935,0.085597,0.093630,0.224817,0.210467,0.066766,0.249194,0.102803
2022-03-07,0.195507,0.251947,0.107833,0.228425,0.138398,0.275930,0.218736,0.230911,0.108842,0.130151,...,0.078882,0.183495,0.130879,0.085597,0.135630,0.202896,0.221810,0.256762,0.061399,0.131400
2022-03-08,0.198365,0.219089,0.060267,0.123045,0.225397,0.275930,0.119492,0.265936,0.105696,0.136305,...,0.259307,0.122920,0.135396,0.234194,0.129117,0.106159,0.221810,0.243581,0.061399,0.210359
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-23,0.137972,0.247531,0.117714,0.076038,0.135675,0.100439,0.133945,0.281660,0.280728,0.074356,...,0.219988,0.246164,0.107220,0.085597,0.121228,0.269260,0.216974,0.122643,0.289465,0.133700
2022-05-24,0.098172,0.243426,0.062975,0.106242,0.116763,0.134868,0.219910,0.215345,0.121632,0.262266,...,0.133712,0.183495,0.080703,0.085597,0.206061,0.229651,0.217994,0.243195,0.088067,0.210359
2022-05-25,0.098247,0.243426,0.256568,0.123576,0.134611,0.074119,0.219910,0.093857,0.108842,0.074356,...,0.219988,0.183495,0.114851,0.126226,0.206061,0.224817,0.106188,0.114860,0.289465,0.213763


In [41]:
print(RPS4.mean(axis=1).mean())

0.15834803367713188


In [42]:
print(RPS4.mean(axis=1))

date
2022-03-02    0.162221
2022-03-03    0.163418
2022-03-04    0.152615
2022-03-07    0.154842
2022-03-08    0.156411
                ...   
2022-05-23    0.161741
2022-05-24    0.155645
2022-05-25    0.156620
2022-05-26    0.158333
2022-05-27    0.159621
Length: 62, dtype: float64


In [43]:
#save results
RPS4.to_excel('Q1-Gradient Boosting_next_month_prediction.xlsx')